<a href="https://colab.research.google.com/github/manuelarguelles/DEMO_ETL_EXCELS_COLLAB_PYTHON_POSTGRESSQL/blob/main/DEMO_ETL_EXCEL_COLLAB_PYTHON_POSTGRESS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!ls -la


total 60
drwxr-xr-x 1 root root 4096 Jul 10 00:12 .
drwxr-xr-x 1 root root 4096 Jul 10 00:01 ..
drwxr-xr-x 4 root root 4096 Jul  8 17:47 .config
-rw-r--r-- 1 root root 5657 Jul 10 00:01 diario_1.xlsx
-rw-r--r-- 1 root root 5666 Jul 10 00:01 diario_2.xlsx
-rw-r--r-- 1 root root 5661 Jul 10 00:01 diario_3.xlsx
-rw-r--r-- 1 root root 5662 Jul 10 00:01 diario_4.xlsx
-rw-r--r-- 1 root root  144 Jul 10 00:12 .env
drwxr-xr-x 1 root root 4096 Jul  8 17:47 sample_data
-rw-r--r-- 1 root root 5475 Jul 10 00:01 sku_maestro.xlsx


In [4]:
# Instalar dependencias
!pip install python-dotenv psycopg2-binary

# Importar dependencias
import os
from dotenv import load_dotenv
import psycopg2

# Cargar variables desde .env
load_dotenv(".env")

# Conectar a PostgreSQL
conn = psycopg2.connect(
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    dbname=os.getenv("DB_NAME")
)
cur = conn.cursor()

# Crear tabla ventas_diarias
crear_tabla_query = '''
CREATE TABLE IF NOT EXISTS ventas_diarias (
    sku_local_fecha TEXT PRIMARY KEY,
    sku_local TEXT,
    nombre_producto TEXT,
    fecha DATE,
    cantidad INTEGER,
    precio NUMERIC
);
'''

cur.execute(crear_tabla_query)
conn.commit()

# Cerrar conexión
cur.close()
conn.close()

print("Tabla creada exitosamente.")


Tabla creada exitosamente.


In [12]:
# Instalar librerías necesarias (solo la primera vez)
!pip install -q python-dotenv psycopg2-binary sqlalchemy openpyxl

# Importaciones
import pandas as pd
import glob
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
from urllib.parse import quote_plus

# Cargar variables de entorno desde .env
load_dotenv("/content/.env")

host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
database = os.getenv("DB_NAME")

# Ruta donde están los archivos Excel
ruta_archivos = '/content'
archivos = sorted(glob.glob(f'{ruta_archivos}/diario_*.xlsx'))

# Leer y unir archivos diarios
df_consolidado = pd.concat([pd.read_excel(archivo) for archivo in archivos], ignore_index=True)

# Crear columna SKU_LOCAL_FECHA
df_consolidado['SKU_LOCAL_FECHA'] = df_consolidado['SKU_LOCAL'] + "_" + df_consolidado['FECHA'].astype(str)

# Leer maestro
df_maestro = pd.read_excel(f'{ruta_archivos}/sku_maestro.xlsx')

# Cruce con maestro
df_final = df_consolidado.merge(df_maestro, on='SKU_LOCAL', how='left')

# Reordenar columnas
df_final = df_final[['SKU_LOCAL_FECHA', 'SKU_LOCAL', 'NOMBRE_PRODUCTO', 'FECHA', 'CANTIDAD', 'PRECIO']]

# 🔍 Validación previa
print("🔍 Vista previa de los datos que se enviarán a PostgreSQL:")
display(df_final.head())

# Convertir columnas a minúsculas
df_final.columns = df_final.columns.str.lower()

# Codificar contraseña
from urllib.parse import quote_plus
password_encoded = quote_plus(password)

# Conexión a PostgreSQL
engine = create_engine(f'postgresql://{user}:{password_encoded}@{host}:{port}/{database}')

# Enviar datos corregidos
df_final.to_sql('ventas_diarias', con=engine, if_exists='append', index=False)

print("✅ Datos cargados exitosamente.")




🔍 Vista previa de los datos que se enviarán a PostgreSQL:


,SKU_LOCAL_FECHA,SKU_LOCAL,NOMBRE_PRODUCTO,FECHA,CANTIDAD,PRECIO
0,SKU001_2025-07-01,SKU001,Producto A,2025-07-01,4,454
1,SKU003_2025-07-01,SKU003,Producto C,2025-07-01,2,139
2,SKU001_2025-07-01,SKU001,Producto A,2025-07-01,10,309
3,SKU003_2025-07-01,SKU003,Producto C,2025-07-01,8,310
4,SKU003_2025-07-01,SKU003,Producto C,2025-07-01,12,448


✅ Datos cargados exitosamente.
